In [ ]:
import warnings
warnings.filterwarnings("ignore")
from brian2 import *
numpy.set_printoptions(threshold=numpy.nan)
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from multiprocessing import *
from scipy.stats import norm

In [ ]:
__all__ = ['generate_random_mcalpine_et_al_2001_bds']

def fixup(s):
    s = s.replace(',', '.')
    s = s.split('\n')
    s = [map(float, w.split('    ')) for w in s if w]
    f, bitd = zip(*s)
    f = array(f)*kHz
    bitd = array(bitd)*usecond
    return f, bitd

fig_2a_means = '''
0,09455    707,10712
0,16542    520,84442
0,23318    361,37778
0,29635    277,76535
0,35333    232,09654
0,41458    182,66420
0,46000    163,59335
0,51884    205,06943
0,57556    148,14299
0,61844    113,97392
0,68096    147,91190
0,75553    117,48437
0,80553    121,18188
0,99987    109,52809
'''

fig_2a_means_plus_stds = '''
0,09879    1125,42432
0,19757    819,93372
0,30073    604,84766
0,39557    412,23495
0,49462    412,60233
0,59540    333,41052
0,68949    242,79839
0,78939    307,37531
0,89622    250,80063
0,97863    201,73302
1,09955    209,49567
1,23526    228,61478
1,34885    179,54718
1,75320    191,33490
'''

_, mean_bitd = fixup(fig_2a_means)
f, bitd_mean_plus_std = fixup(fig_2a_means_plus_stds)
std_bitd = bitd_mean_plus_std-mean_bitd

def generate_random_mcalpine_et_al_2001_bds(cf,N=360,std_factor=1.0):
    fmid = 0.5*(f[1:]+f[:-1])
    I = digitize(cf, fmid)
    mu = mean_bitd[I]*2*cf[0]*180.0
    sigma = std_bitd[I]*std_factor*2*cf[0]*180.0
    x_axis = np.arange(-180, 180, 360/N)
    dist_bipd=exp(-(mu-x_axis)**2/(2*sigma**2))+ exp(-(-mu-x_axis)**2/(2*sigma**2))
    dist_bipd_recentered=np.concatenate((dist_bipd[int(N/2):int(N)],dist_bipd[0:int(N/2)]))
    #plot(dist_bipd_recentered)
    #show()
    return dist_bipd_recentered


In [ ]:
def hemispheric_decoder(activity_vec):
    half_dim=int(0.5*activity_vec.shape[0])

    act_ipsi=activity_vec[0:half_dim]
    act_contra=activity_vec[half_dim:activity_vec.shape[0]]
    
    sum_ipsi=sum(act_ipsi)

    sum_contra=sum(act_contra)

    ratio=(sum_ipsi-sum_contra)/sum(activity_vec)

    return ratio
    

In [ ]:
def static_ipd_dens_plots_generation(Nx,Ny,fm,fc,ipd,tc='Cosine with exponent',k=4,sigma=0.5,procnum=0,return_dict=None):
    fm=fm*Hz
    fc=fc*Hz
    n=3
    t = linspace(0, 1, Nx)*(n/fm)
    bipd = linspace(0, 2*pi, Ny)%(2*pi) 
    T, BIPD = meshgrid(t, bipd) 
    A = 0.5*(1-cos(2*pi*fm*T))   
    
    PHI =ipd
    if tc=='Cosine with exponent':
        TC=(cos((BIPD-PHI)/2))**k
    else:
        TC = (1.0/3)*(exp(-(BIPD-PHI)**2/(2*sigma**2))+
                   exp(-(BIPD-PHI+2*pi)**2/(2*sigma**2))+
                   exp(-(BIPD-PHI-2*pi)**2/(2*sigma**2)))

    layer1 = TC*A
    
    h_bitd=generate_random_mcalpine_et_al_2001_bds([fc],Ny, std_factor=1.0)
    layer_weighted=np.multiply(layer1,h_bitd[...,np.newaxis])
    return_dict[procnum] =layer_weighted

In [ ]:
def training(Nx,Ny,fc,num_training,tc,sigma,k):
    ipds=np.linspace(0,2*pi,num_training)    
    num_fm=5
    training_patts=np.zeros((num_fm,num_training,Ny,Nx))
    proc=[]
    manager = Manager()
    return_dict = manager.dict()

    for i in xrange(num_fm):
        for j in xrange(num_training):
            proc.append(Process(target=static_ipd_dens_plots_generation,args=(Nx,Ny,2**(i+2),fc,ipds[j],tc,k,sigma,i*num_training+j,return_dict,)))
        
    for i in xrange(num_fm):
        for j in xrange(num_training):
            proc[i*num_training+j].start()
        
    for i in xrange(num_fm):
        for j in xrange(num_training):
            proc[i*num_training+j].join()
        
    for i in xrange(num_fm):
        for j in xrange(num_training):
            training_patts[i,j,:,:]= return_dict[i*num_training+j]
            
    return training_patts

In [ ]:
def dyn_ipd_dens_plot_generation(Nx,Ny,fm,fc,tc='Cosine with exponent',sigma=0.5,k=4):
    fm=fm*Hz
    fc=fc*Hz
    n=3
    t = linspace(0, 1, Nx)*(n/fm)
    bipd = linspace(0, 2*pi, Ny)%(2*pi) 
    T, BIPD = meshgrid(t, bipd) 
    A = 0.5*(1-cos(2*pi*fm*T))
    
    PHI = (2*pi*fm*t)%(2*pi)
    
    if tc=='Cosine with exponent':
        TC=(cos((BIPD-PHI)/2))**k
    else:
        TC = (1.0/3)*(exp(-(BIPD-PHI)**2/(2*sigma**2))+
                   exp(-(BIPD-PHI+2*pi)**2/(2*sigma**2))+
                   exp(-(BIPD-PHI-2*pi)**2/(2*sigma**2)))
   
    layer1 = TC*A
                      
    h_bitd=generate_random_mcalpine_et_al_2001_bds([fc],Ny, std_factor=1.0)
    layer_weighted=np.multiply(layer1,h_bitd[...,np.newaxis])
    #imshow(layer_weighted)
    #show()
    return layer1,layer_weighted

In [ ]:
def testing_dyn_ipd(Nx,Ny,fc,tc,sigma,k):   
    num_fm=5
    testing_patts=np.zeros((num_fm,Ny,Nx))

    for i in xrange(num_fm):
        testing_patts[i,:]=dyn_ipd_dens_plot_generation(Nx,Ny,2**(i+2),fc,tc,sigma,k)[1]
        
            
    return testing_patts
    

In [ ]:
def testing(Ny,num_training,test_img,training_patts,similarity_method,decision_window):
    num_fm=5
    scores=np.zeros((num_fm,num_training))
    results=np.zeros(num_fm)
    hemi_ratio_tests=np.zeros(num_fm)
    hemi_ratio_trains=np.zeros((num_fm,num_training))
    results_hemi=np.zeros(num_fm)
    sum_tests=np.zeros((num_fm,Ny))
    sum_train_dec_winds=np.zeros((num_fm,num_training,Ny))
    abs_diff_ratio=np.zeros((num_fm,num_training))
         
        
    if similarity_method=='Euclidean distance':
        for i in xrange(num_fm):
            for j in xrange(num_training):
                scores[i,j]=np.linalg.norm(test_img[i][:,decision_window[0]:decision_window[1]]-training_patts[i,j,:,decision_window[0]:decision_window[1]])

        for i in xrange(num_fm):
            for j in xrange(num_training): 
                results[i]=argmin(scores[i,:])*360/(num_training-1)
                
    else:
        
        for i in xrange(num_fm):
            for j in xrange(num_training):
                sum_test=sum(test_img[i][:,decision_window[0]:decision_window[1]],1)
                sum_train=sum(training_patts[i,j,:,decision_window[0]:decision_window[1]],1)
                scores[i,j]=np.dot(sum_test,sum_train)
                scores[i,j]=scores[i,j]/(np.linalg.norm(sum_test)*np.linalg.norm(sum_train))
       
        for i in xrange(num_fm):
            for j in xrange(num_training): 
                results[i]=argmax(scores[i,:])*360/(num_training-1)
                
    ### Hemispheric decoder
    for i in xrange(num_fm):
            sum_tests[i]=sum(test_img[i][:,decision_window[0]:decision_window[1]],1)
            hemi_ratio_tests[i]=hemispheric_decoder(sum_tests[i])
            
    for i in xrange(num_fm):
        for j in xrange(num_training):
            sum_train_dec_winds[i,j,:]=sum(training_patts[i,j,:,decision_window[0]:decision_window[1]],1)
            hemi_ratio_trains[i,j]=hemispheric_decoder(sum_train_dec_winds[i,j,:])
            
    for i in xrange(num_fm):
        for j in xrange(num_training):
            
            abs_diff_ratio[i,j]=hemi_ratio_tests[i]-hemi_ratio_trains[i,j]
            
    results_hemi=argmin(abs_diff_ratio,1)*360.0/(num_training-1)
    
                            
    return scores, results, results_hemi

In [ ]:
def comparison_decoder(Nx,Ny,fc,num_training,tc,sigma,k, similarity_method,decision_window):
    
    training_patts=training(Nx,Ny,fc,num_training,tc,sigma,k)

    testing_patts=testing_dyn_ipd(Nx,Ny,fc,tc,sigma,k)
    res=testing(Ny,num_training,testing_patts,training_patts,similarity_method,decision_window)
 
    
    figure(figsize=(18,12))
    fm=4
    dynIPD4=dyn_ipd_dens_plot_generation(Nx,Ny,fm,fc,tc,sigma,k)
    gs = GridSpec(3, 3) # height_ratios=[2, 2])
    subplot(gs[0,0])
    title('Density plot, fm= 4 Hz before BIPD weighting')
    imshow(dynIPD4[0],origin='lower left', interpolation='nearest', aspect='auto')
    colorbar()
    subplot(gs[0,1])
    dist_bipd= generate_random_mcalpine_et_al_2001_bds([fc*Hz],Ny,std_factor=1.0)
    title('McAlpine distribution of BIPD when fc='+str(fc)+' Hz')
    plot(dist_bipd)
    xlabel('BIPD (deg)')
    ylabel('Normalized frequency')
    subplot(gs[0,2])
    title('Density plot, fm= 4 Hz after BIPD weighting')
    imshow(dynIPD4[1],origin='lower left', interpolation='nearest', aspect='auto')
    colorbar()
    xlabel('BIPD (deg)')
    ylabel('Normalized frequency')
    subplot(gs[1,0])
    n=int(num_training/4.0)
    title('Example training data, ipd=' + str(n*360.0/(num_training-1))+ 'deg, fm= 4 Hz')
    imshow(training_patts[0,n],origin='lower left', interpolation='nearest', aspect='auto')
    colorbar()
    xlabel('BIPD (deg)')
    ylabel('Normalized frequency')
    
    subplot(gs[1,1])
    title('Decision window, fm= 4 Hz')
    imshow(testing_patts[0][:,decision_window[0]:decision_window[1]],origin='lower left', interpolation='nearest', aspect='auto')
    colorbar()
    xlabel('BIPD (deg)')
    ylabel('Normalized frequency')
    
    subplot(gs[1,2])
    title('Scoring of the dyn IPD with the trining pattern')
    imshow(transpose(res[0]),origin='lower left', interpolation=None, aspect='auto',extent=[4,64,0,360])
    ylabel('BIPD (deg)')
    xlabel('Modulation frequency')
    colorbar()
    
    fm_array=[4,8,16,32,64]*Hz
    subplot(gs[2,0:1])
    title('Phase_fm curve')
    plot(fm_array, res[1], '-o', label=str(similarity_method))
    plot(fm_array, res[2], '-o', label='Hemispheric decoder')
    plot(fm_array, [37, 40, 62, 83, 115], '--r', label='Data')
    legend(loc='upper left')
    grid()
    ylim(0, 180)
    xlabel('Modulation frequency (Hz)')
    ylabel('Extracted phase (deg)')
    
    
    
    tight_layout()
    
w=interactive(comparison_decoder,
     Nx=widgets.IntSlider(min=100, max=5000, step=100, value=1000,description='Number of pixel on the time axis'),
     Ny=widgets.IntSlider(min=100, max=5000, step=100, value=360,description='Number of pixel on the BIPD axis'),
     fc=widgets.IntSlider(min=0, max=1000, step=100, value=500,description='Carrier frequency (Hz)'),
     num_training=widgets.IntSlider(min=10, max=1000, step=10, value=20,description='Number of training samples'),
     tc=widgets.Dropdown(options=['Cosine with exponent', 'Exponential'],value='Cosine with exponent',description='Tuning Curve'),
     sigma=widgets.FloatSlider(min=0,max=2,step=0.25,value=0.5,description='Width tuning curve when choosing "Exponential"'),
     k=widgets.IntSlider(min=0, max=50, step=2, value=4,description='Exponent cosine k'),
     similarity_method=widgets.Dropdown(options=['Euclidean distance','Summing and dot product'],value='Euclidean distance',description='Similarity distance between the 2 decision windows'),
     decision_window=widgets.IntRangeSlider(value=[0,1000],min=0,max=1000,step=1,description='Decision window'))

# Improve layout
for child in w.children:
    if isinstance(child, widgets.ValueWidget):
        child.layout.width = '100%'
        child.style = {'description_width': '30%'}
        child.continuous_update = False

display(w)
    
    
